In [1]:
!nvcc --version
!pip install bitsandbytes-cuda112 bitsandbytes # based on nvcc --version output

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import gc
import torch


tokenizer = AutoTokenizer.from_pretrained('dicta-il/dictalm-7b-instruct')

In [ ]:
gc.collect()

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained('dicta-il/dictalm-7b-instruct', quantization_config=quantization_config, trust_remote_code=True)

In [8]:
model.eval()

with torch.inference_mode():
    prompt = 'מה אתה חושב על המצב במדינת ישראל:\n'
    kwargs = dict(
        inputs=tokenizer(prompt, return_tensors='pt').input_ids.to(model.device),
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.75,
        max_length=100,
        min_new_tokens=5
    )

    print(tokenizer.batch_decode(model.generate(**kwargs), skip_special_tokens=True))

['מה אתה חושב על המצב במדינת ישראל:\n\nאני חושב שאנחנו צריכים לנקוט עמדה חזקה יותר לגבי מה שקורה כאן. אני חושב שחשוב לנקוט עמדה חזקה יותר כדי לתרום לדיון.\n']
